# Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px

from pose import *
from df_to_db import *
import warnings
warnings.filterwarnings('ignore') # Suppressing warnings

# Running main

In [ ]:
reps = {}
stats_dict = {}
main_pose(stats_dict,reps,"bicep_curl","right",100)

# Saving relevant values as lists

In [ ]:
rep_no = list(range(1, reps['count']+1))

time = list(reps['time'].values())[:reps['count']]
time = [round(x,1) for x in time] # Rounding values to single decimal point
time_label = [f'{str(x)}s' for x in time]

correct_form = list(reps['correct_form'].values())[:reps['count']]
correct_form = [round(x,1) for x in correct_form]
correct_form_label = [f'{str(x)}s' for x in correct_form]

wrong_form = list(reps['wrong_form'].values())[:reps['count']]
wrong_form = [round(x,1) for x in wrong_form]
wrong_form_label = [f'{str(x)}s' for x in wrong_form]

# Storing lists in a pandas dataframe

In [ ]:
data = [rep_no, time, correct_form, wrong_form]
columns = ['rep_no', 'time', 'correct_form', 'wrong_form']

In [ ]:
data = pd.DataFrame(data, columns).transpose()

In [ ]:
data

# Saving into database

In [ ]:
df = dataframer(reps)

In [2]:
databaser()

# Making another dataframe to save form of each frame

In [ ]:
form_df = pd.DataFrame(reps['frame_form']).transpose()
form_df.columns=['Rep no.','Form']
form_df['Frame']=form_df.index+1
form_df['Ones']=np.ones([reps['frame']])
form_df

# X and Y axis settings

In [ ]:
xaxis=dict(showline=True,
           linewidth=2,
           linecolor='black',
           mirror=True, # To create border outline
           dtick=1) # Setting x ticks to show each rep count

yaxis=dict(showline=True,
           linewidth=2,
           linecolor='black',
           mirror=True)

xaxis2=dict(showticklabels=False,
           showline=True,
           linewidth=2,
           linecolor='black',
           mirror=True) # To create border outline

yaxis2=dict(showticklabels=False,
           showline=True,
           linewidth=2,
           linecolor='black',
           mirror=True)

# Time vs. Reps iplot using plotly

In [ ]:
fig1= go.Figure()

fig1.add_trace(go.Scatter(x=data['rep_no'],
                         y=data['time'],
                         text=time_label,
                         hoverinfo='text',
                         line=dict(color='darkviolet',
                                   width=3),
                         stackgroup='one' # To fill color beneath the plotted area
                        ))

fig1.update_layout(title='Time spent doing each rep',
                  xaxis_title='Rep no.',
                  yaxis_title='Time (s)',
                  xaxis=xaxis,
                  yaxis=yaxis,
                  plot_bgcolor='#EAEAF2',
                  font_size=15
                 )

fig1.show()

# Time vs. Reps iplot comparing correct form to wrong form

In [ ]:
fig2 = go.Figure()

fig2.add_trace(go.Scatter(
    name='Wrong form',
    x=rep_no,
    y=wrong_form,
    text=wrong_form_label,
    hoverinfo='text',
    mode='lines',
    line=dict(width=3,
              color='red'),
    stackgroup='one'
))

fig2.add_trace(go.Scatter(
    name='Correct form',
    x=rep_no,
    y=correct_form,
    text=correct_form_label,
    hoverinfo='text',
    mode='lines',
    line=dict(width=3,
              color='rgb(111, 231, 219)'),
    stackgroup='one'
))

fig2.update_layout(title='Correct form vs. Wrong form with rep progression',
                   xaxis_title='Rep no.',
                   yaxis_title='Time (s)',
                   xaxis=xaxis,
                   yaxis=yaxis,
                   plot_bgcolor='#EAEAF2',
                   font_size=15
                  )

fig2.show()

# Barplot comparing cumulative time spent doing correct and wrong form

In [ ]:
total_correct =round(sum(correct_form), 1)
total_wrong = round(sum(wrong_form), 1)

x_bar = ['Correct form', 'Wrong form']
y_bar = [total_correct, total_wrong]
y_label = [f'{str(x)}s' for x in y_bar]

form_colors = ['rgba(111, 231, 219, 0.5)', 'rgba(255,0,0,0.5)'] # Using rgba to set different opacity for marker and marker line
form_line_colors = ['rgba(111, 231, 219, 1)', 'rgba(255,0,0,1)']

In [ ]:
fig3 = go.Figure([go.Bar(x=x_bar, 
                         y=y_bar,
                         text=y_label, 
                         hoverinfo='x+text',
                         textposition='outside',
                         marker=dict(color=form_colors,
                                     line=dict(color=form_line_colors,
                                               width=3))
                        )])

fig3.update_layout(title='Correct form vs. Wrong form (Cumulative)',
                   yaxis_title='Time (s)',
                   plot_bgcolor='#EAEAF2',
                   font_size=15
                  )

fig3.show()

# Pie chart comparing cumulative time spent doing correct and wrong form

In [ ]:
fig4 = go.Figure(data=[go.Pie(labels=x_bar,
                              values=y_bar)])

fig4.update_traces(hoverinfo='label+percent', 
                   text=y_label,
                   textinfo='text', 
                   textfont_size=20,
                   marker=dict(colors=form_colors, 
                               line=dict(color=form_line_colors, 
                                         width=3))
                  )

fig4.show()

# Rug plot showing continuous form in each frame

In [ ]:
fig5 = px.bar(data_frame=form_df,
             x='Frame',
             y='Ones',
             hover_data={'Rep no.':True,
                        'Ones':False,
                        'Frame':False,
                        'Form':False},
             labels={'Ones':'','Frame':'Reps'},
             color='Form',
             color_discrete_sequence=["red", "rgb(111, 231, 219)"],
             height=400,
            )

fig5.update_layout(title='Continuous form with rep progression',
                   xaxis=xaxis2,
                   yaxis=yaxis2,
                   plot_bgcolor='#EAEAF2',
                   font_size=15,
                   legend_title_text='',
                   bargap=0, # gap between bars of adjacent location coordinates.
                  )
fig5.show()